<a href="https://colab.research.google.com/github/Vadiman728/forecast-of-securities-prices/blob/main/%D0%9F%D1%80%D0%BE%D0%B3%D0%BD%D0%BE%D0%B7_%D1%86%D0%B5%D0%BD%D1%8B_%D1%86%D0%B5%D0%BD%D0%BD%D1%8B%D1%85_%D0%B1%D1%83%D0%BC%D0%B0%D0%B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Наша модель будет решать задачу регрессии цен закрытия торгов по ценным бумагам. Сей датасет легко предоставляет mfd.ru . Будет несколько бумаг, по ним будет предварительная обработка и, чтобы не тратить время на поиск оптимальной архитектуры- воспользуемся предобученными моделями и внедрим выбор количества эпох

В итоге мы получаем: Пользователь выбирает между бумагами, затем между моделями, выбирает на какой промежуток сделать прогноз и количество эпох. На выходе получаем прогноз цены на выбранный диапазон, и график цен. Некий финансовый помощник

Сначала скачиванием текстовый файл, импортируем его в Excel, выставляем разделители и сохраняем это чудо в диск и импортируем

In [ ]:
!pip install streamlit
!pip install catboost
!pip install yadisk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 kB 2.7 MB/s eta 0:00:00


In [ ]:
%%writefile main.py
# импортируем основную библиотеку
import streamlit as st
# импортируем предобученные модели
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from catboost import CatBoostRegressor
# Импортируем слои и оптимизацию для настраиваемой модели
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
# библиотеки для работы с загрузкой датасета и самим датасетом
import pandas as pd
from sklearn.model_selection import train_test_split
import requests
from urllib.parse import urlencode
import numpy as np

# иные библиотеки
import time
import matplotlib.pyplot as plt
import io

'Для начала выберем, хотите ли вы довериться нашей архитектуре или довериться архитектуре предзагруженных моделей'
choice_list= ['Предобученная модель', 'собственная архитектура']
choice = st.selectbox('Make your choice: ', choice_list)
'Your choice is ', choice

if choice == 'Предобученная модель':
  st.header("Обучаем модель")
  # выбираем модель
  st.subheader("Выберите предобученую модель")
  'Обучение будет вестись на заранее предобученных моделях'
  models_dict = {'Linear Regression': LinearRegression(),
                'SVR': SVR(C= 1.4 ),
                'Random Forest Regressor': RandomForestRegressor(max_depth=6) ,
                'Gradient Boosting Regressor': GradientBoostingRegressor(loss='absolute_error'),
                'Gaussian Process Regressor': GaussianProcessRegressor(),
                'CatBoost Regression': CatBoostRegressor(iterations=500, learning_rate=0.1, depth=12) }  # список моделей
  model_show = st.selectbox('Выберите модель из списка', list(models_dict.keys()))
  'Вы выбрали', model_show
  model = models_dict[model_show]
# пишем LSTM модель- модель, делающая прогнозы, используя временные ряды- то, что нейродоктор прописал
elif choice == 'собственная архитектура':
  time_step = 60

  model = Sequential()
  model.add(LSTM(units=50, return_sequences=True, input_shape=(time_step, 1), name = 'lstm1'))
  model.add(Dropout(0.2, name = 'dropout1'))
  model.add(LSTM(units=50, return_sequences= False, name = 'lstm2'))
  model.add(Dropout(0.3, name = 'dropout2'))
  model.add(Dense(units=1))

  model.compile(optimizer = 'adam', loss= 'mse')

else: 'Ошибка выбора модели или иная ошибка'

# теперь выбираем ценную бумагу
st.subheader("Выберите ценную бумагу, по которой модель будет предсказывать")
'Выберите бумагу, по которой хотите сделать прогноз. Наши базы постоянно не обновляются'
paper_dict = {
              "Лукойл": 'https://disk.yandex.ru/d/NABhGrzq6ifIjg',
              "Сегежа груп": 'https://disk.yandex.ru/d/UrK1aBs4SUJ9Bw',
              "РусАл": 'https://disk.yandex.ru/d/D_ePLrlreG94yw',
              "S&P 500": 'https://disk.yandex.ru/d/lebCIPjRwgiRaw',
              "Золото": 'https://disk.yandex.ru/d/EO4-JW4y1fjOFA' }
paper_show = st.selectbox('Выберите ценную бумагу из списка:', list(paper_dict.keys()))
'Вы выбрали бумагу', paper_show
if paper_show:

  paper_link = paper_dict[paper_show]
  progress_bar = st.progress(0) # добавим красоты
  # в уроках используется ссылка через yandexcloud, который создан для бизнеса и тд и тп
  # тут же используется костыль, найденный на просторах интернета, с помощью которого
  # можно прогружать данные с собственного диска... притом рабочий
  base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
  public_key = 'https://disk.yandex.ru/d/SauOo61x4VIjAQ'  # Сюда вписываете вашу ссылку
  # Получаем загрузочную ссылку
  final_url = base_url + urlencode(dict(public_key=public_key))
  response = requests.get(final_url)
  download_url = response.json()['href']
  # Загружаем файл и сохраняем его
  download_response = requests.get(download_url)
  with open('dataset.csv', 'wb') as f:   # Здесь укажите нужный путь к файлу
      f.write(download_response.content)
  df = pd.read_csv('/content/dataset.csv', sep= ';')
  df = df.drop(['<TICKER>', '<PER>', '<TIME>','<DATE>'], axis=1)
  if choice == 'Предобученная модель':
    x = df.drop('<CLOSE>', axis = 1)
    y = df['<CLOSE>']
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)
    progress_bar.progress(100) # заканчиваем красоту
    'Датасет готов к использованию'
    # Обучаем модель
    'Начало обучения'
    progress_bar2 = st.progress(0)
    model.fit(x_train, y_train)
    progress_bar2.progress(100)
    "Конец обучения"

  # придется прописать отдельно обработку для LSTM модели

  elif choice == 'собственная архитектура':
    df = df.dropna()
    scaler = MinMaxScaler(feature_range = (0,1))
    scaled_data = scaler.fit_transform(df['<CLOSE>'].values.reshape(-1,1))

    def create_dataset(dataset, time_step=1):
      dataX, dataY = [], []
      for i in range(len(dataset)- time_step - 1):
        a = dataset[i:(i+ time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_step,0])
      return np.array(dataX), np.array(dataY)
    time_step = 60
    X, y = create_dataset(scaled_data, time_step)
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
    x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)
    progress_bar.progress(100) # заканчиваем красоту
    'Датасет готов к использованию'
    'Перед началом обучения необходимо выбрать количество эпох и batch_size'
    EPOCHS = st.slider('Количество эпох')
    'Количество эпох: ', EPOCHS
    BATCH_SIZE = st.slider('batch_size',min_value = 4, max_value = 512, step = 4)
    'Размер батча: ', BATCH_SIZE
    'Начало обучения'
    progress_bar2 = st.progress(0)
    model.fit(x_train, y_train, epochs = EPOCHS, batch_size = BATCH_SIZE, validation_data = (x_test, y_test), verbose =1)
    progress_bar2.progress(100)
    "Конец обучения"

# Прогнозируем на n дней
st.subheader("Анализ модели")
'Здесь вы увидите, насколько сильно предсказанные данные отличаются от истинных'

if choice == 'Предобученная модель':
  x_last = x.iloc[-1].values.reshape(1, -1)  # берем последние данные из x
  predictions = []
  n = 0
  for i in range(20):
      y_pred = model.predict(x_last)
      predictions.append(y_pred[0])
      n = n + 1
      x_last = np.append(x_last[:, 1:], y_pred).reshape(1, -1)  # обновляем x_last новыми предсказаниями

  real_close = y.values[-20:]

  plt.figure(figsize=(20, 10))
  plt.plot(range(1, 21), real_close, marker='o', label='Реальные цены закрытия')
  plt.plot(range(1, 21), predictions, marker='x', label='Предсказанные цены закрытия')
  plt.xlabel('Дни')
  plt.ylabel('Цена закрытия')
  plt.title('Сравнение предсказанных и реальных цен закрытия')
  plt.legend()
  plt.grid(True)
  st.pyplot(plt)

elif choice == 'собственная архитектура':
  loss = model.evaluate(x_test, y_test)
  'Test Loss:' ,loss

  # Предсказание
  predictions = model.predict(x_test)
  predictions = scaler.inverse_transform(predictions)

  # Преобразование обратно в исходный масштаб
  y_test_scaled = scaler.inverse_transform(y_test.reshape(-1, 1))

  # Отображение результатов
  import matplotlib.pyplot as plt

  plt.figure(figsize=(14, 5))
  plt.plot(y_test_scaled, color='blue', label='Actual Stock Price')
  plt.plot(predictions, color='red', label='Predicted Stock Price')
  plt.title('Stock Price Prediction')
  plt.xlabel('Time')
  plt.ylabel('Stock Price')
  plt.legend()
  plt.grid(True)
  st.pyplot(plt)


if choice == 'Предобученная модель':
  st.header('Прогноз по введенным данным')
  'Тут вы можете самостоятельно вручную ввести данные, соблюдая при этом следующие правила:'
  'Вводите столько данных в одну строку, сколько столбцов'
  'Разделяйте данные точкой с запятой ";" '
  'Переход на следующую строку- Shift + Enter БЕЗ ";" в конце'

  # Используем виджет text_area для ввода многострочного текста
  user_data = st.text_area("Введите данные в следующем порядке: Open; High; Low; Vol; Openint ", '''Пример: 123;123;123;123;123;123
  123;123;123;123;123;123''')


  Testdata =f'''
  <OPEN>;<HIGH>;<LOW>;<VOL>;<OPENINT>;<CLOSE>
  {user_data}
  '''
  df_user = pd.read_csv(io.StringIO(Testdata), sep=";")
  df_user
  df_user = df_user.drop['<CLOSE>']
  x_user = df_user.values
  type(x_user)
  user_predict = model.predict(df_user)
  'Предсказание цены закрытия по вашим данным:'
  user_predict

elif choice == 'собственная архитектура':
  'Собственная архитектура предсказывает на 10 дней вперед'
  st.header('Прогноз по введенным данным')
  user_data = st.text_area("Введите данные в следующем порядке: Close", '''Пример: 123\n123''')

  Testdata = f'<CLOSE>\n{user_data}'
  df_user = pd.read_csv(io.StringIO(Testdata), sep=";")

  if len(df_user) < time_step:
      st.error(f"Данных недостаточно для предсказания. Требуется минимум {time_step} записей.")
  else:
      df_user_scaled = scaler.transform(df_user['<CLOSE>'].values.reshape(-1, 1))
      def predict_n_days(model, data, time_step, n_days):
        predictions = []
        last_sequence = data[-time_step:]  # последние time_step данных для старта

        if len(last_sequence) < time_step:
            raise ValueError(f"Размер данных ({len(last_sequence)}) меньше, чем необходимый размер временного шага ({time_step})")

        for _ in range(n_days):
            input_data = last_sequence.reshape((1, time_step, 1))
            pred = model.predict(input_data)
            predictions.append(pred[0, 0])
            last_sequence = np.append(last_sequence[1:], pred[0, 0]).reshape(-1, 1)

        return np.array(predictions)
      predictions = predict_n_days(model, df_user_scaled, time_step, 10)
      predictions = scaler.inverse_transform(predictions.reshape(-1, 1))
      st.write("Предсказания на 10 дней вперед: ", predictions)

Writing main.py


In [ ]:
!streamlit run main.py --server.address=localhost >/content/logs.txt & ssh -o "StrictHostKeyChecking no" -R 80:localhost:8501 serveo.net

Forwarding HTTP traffic from https://897738245c2a0fee656c9b4ab9e9ba52.serveo.net
HTTP request from 37.112.183.131 to https://897738245c2a0fee656c9b4ab9e9ba52.serveo.net/
HTTP request from 37.112.183.131 to https://897738245c2a0fee656c9b4ab9e9ba52.serveo.net/static/media/SourceSansPro-Regular.0d69e5ff5e92ac64a0c9.woff2
HTTP request from 37.112.183.131 to https://897738245c2a0fee656c9b4ab9e9ba52.serveo.net/static/media/SourceSansPro-SemiBold.abed79cd0df1827e18cf.woff2
HTTP request from 37.112.183.131 to https://897738245c2a0fee656c9b4ab9e9ba52.serveo.net/static/media/SourceSansPro-Bold.118dea98980e20a81ced.woff2
HTTP request from 37.112.183.131 to https://897738245c2a0fee656c9b4ab9e9ba52.serveo.net/static/css/main.3aaaea00.css
HTTP request from 37.112.183.131 to https://897738245c2a0fee656c9b4ab9e9ba52.serveo.net/static/js/main.7994a814.js
HTTP request from 37.112.183.131 to https://897738245c2a0fee656c9b4ab9e9ba52.serveo.net/_stcore/health
HTTP request from 37.112.183.131 to https://897

In [ ]:
# задаем количество эпох
st.subheader("Выберите количество эпох")
st.write("Учитывайте, что при бОльшем количестве модель будет переобучаться. Рекомендовано 20-40 эпох")
EPOCHS = st.slider('количество эпох:', min_value=0, max_value=250)
st.write('Вы выбрали ', EPOCHS, "эпох")

# задаем batch_size
st.subheader(" Выберите значение 'batch_size'")
'batch_size- размер данных, одновременно подаваемых на обучение в течение эпохи. Это значение влияет на скорость и на качество обучения'
BATCH_SIZE = st.slider('количество эпох:', min_value=0, max_value=256, step = 4)
"Размер батча составляет: ", BATCH_SIZE